In [1]:
with open('english_sentences.txt', 'r') as f:
    engData = f.readlines()

In [2]:
engData[0]

'however paes who was partnering australias paul hanley could only go as far as the quarterfinals where they lost to bhupathi and knowles\n'

In [3]:
english_sentences = [sent.rstrip('\n') for sent in engData]

In [4]:
english_sentences[0]

'however paes who was partnering australias paul hanley could only go as far as the quarterfinals where they lost to bhupathi and knowles'

In [5]:
text = ''

for sentence in english_sentences:
    text += sentence+' '

text = text.strip()

In [6]:
tokens = list(text.encode('utf-8'))

In [7]:
tokens[0]

104

In [8]:
len(tokens)

9466581

In [9]:
vocab_size = 55000
merges = {}
# from numba import jit
# @jit(target_backend='cuda')
def get_stats(ids):
    count = {}

    for pair in zip(ids, ids[1:]):
        count[pair] = count.get(pair, 0)+1

    return count

# @jit(target_backend='cuda')
def merge(ids, pair, idx):
    i = 0
    newids = []

    while i < len(ids):
        if(i < len(ids)-1 and ids[i] == pair[0] and ids[i+1] == pair[1]):
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [10]:
merge([1,2,5,3,4,3], (3,4), 89)

[1, 2, 5, 89, 3]

In [11]:
num_merges = vocab_size - 256
ids = list(tokens)

for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key = stats.get)

    idx = 256+i

    ids = merge(ids, pair, idx)

    merges[pair] = idx

In [40]:
vocab = {idx:bytes([idx]) for idx in range(256)}

for (p0,p1), idx in merges.items():
    vocab[idx] = vocab[p0]+vocab[p1]

In [41]:
engMerges = {str(key):val for key,val in merges.items()}

In [42]:
import json
with open('English_Merges.json', 'w') as f:
    json.dump(engMerges, f, indent = 4)

In [43]:
len(vocab)

55000

In [44]:
vocab[1]

b'\x01'

In [45]:
len(set(list(vocab.keys())))

55000

In [46]:
index_to_english = {key:str(val) for key,val in vocab.items()}
english_to_index = {str(val):key for key,val in vocab.items()}

In [47]:
with open('index_to_english.json', 'w') as f:
    json.dump(index_to_english, f, indent=4)

with open('english_to_index.json', 'w') as ff:
    json.dump(english_to_index, ff, indent = 4)

In [48]:
len(english_to_index), len(index_to_english)

(55000, 55000)

In [49]:
s = []
for k,v in vocab.items():
    s.append(v)

In [50]:
len(s), len(set(s))

(55000, 55000)

In [56]:
bytes([102,109]), bytes([104,109]).decode('utf-8')

(b'fm', 'hm')